In [1]:
% when running localy
path_to_spm = '/home/remi/matlab/SPM/spm12/';
addpath(path_to_spm)

spm('defaults', 'FMRI');

In [2]:
tr = 1.0;  % repetition time is 1 second
nb_scans = 128;  % the acquisition comprises 128 scans

start_dir = pwd;

ouput_dir = '../ouput/octave/design_matrix';
spm_mkdir(ouput_dir);

Will be using the `spm_run_fmri_spec` function via the SPM batch to create the design matrix.

In [3]:
function cond = set_condition(name, onsets, durations)
    cond.name = name;
    cond.onset = onsets;
    cond.duration = durations;
    cond.tmod = 0;
    cond.pmod = struct('name', {}, 'param', {}, 'poly', {}); % parametric modulation
    cond.orth = true(); % orthogonalize
end

In [4]:
function motion_regressors = simulate_motion(nb_scans)

    % The 6 parameters correspond to three translations and three rotations 
    % describing rigid body motion
    extra_reg_names = {'tx', 'ty', 'tz', 'rx', 'ry', 'rz'};

    for iReg = 1:numel(extra_reg_names)
        motion_regressors(iReg).name = extra_reg_names{iReg};

        # We simulate motion parameters observed with fMRI acquisitions
        motion_regressors.val = cumsum(randn(nb_scans, 1), 1);
    end

end

In [5]:
function design = set_design(ouput_dir, tr, nb_scans, conditions, extra_reg)
    design.dir = cellstr(ouput_dir);

    design.timing.units = 'secs';
    design.timing.RT = tr;
    
    design.bases.hrf.derivs = [0 0];

    design.sess.nscan = nb_scans;
    design.sess.cond = conditions;
    design.sess.hpf = 128;
end

## Event related model with HRF

In [6]:
% these are the types of the different conditions
% with their: names, events onset time, events onset duration

conditions(1) = set_condition('c1', [30, 70, 100], [1, 1, 1]);
conditions(2) = set_condition('c2', [10, 30, 90], [1, 1, 1]);     
conditions(3) = set_condition('c3', [30, 40, 60], [1, 1, 1]);  

extra_reg = simulate_motion(nb_scans);

motion_regressors =

  scalar structure containing the fields:

    name = tx
    val =

        0.14830
        0.30962
        0.51088
        1.00918
        1.73400
        2.53478
        2.20961
        3.22909
        4.78267
        3.87775
        2.11562
        2.11139
        3.18867
        3.35407
        4.24608
        5.98681
        5.51586
        4.74662
        3.75563
        3.76334
        3.27121
        4.79365
        1.52979
        2.12948
        2.57205
        3.02122
        3.80327
        2.69669
        1.29496
       -0.14449
       -1.01204
       -0.60208
        0.77348
       -0.51679
       -0.92008
       -1.66259
       -2.95250
       -3.34562
       -2.04026
       -1.59771
       -1.95801
       -4.25799
       -4.63316
       -5.58122
       -4.52448
       -3.04744
       -3.05578
       -4.43771
       -3.51280
       -3.22328
       -2.03213
       -0.23326
       -2.24029
       -2.01057
       -3.72913
       -4.59270
       -5.41178

In [7]:
design = set_design(ouput_dir, tr, nb_scans, conditions);
matlabbatch{1}.spm.stats.fmri_design = design;

In [8]:
spm_jobman('run', matlabbatch);
cd(start_dir)

Initialising batch system... mv: cannot move '/home/remi/matlab/SPM/spm12/toolbox/.' to '/home/remi/matlab/SPM/spm12/toolbox/MACS/.': Device or resource busy
done.


------------------------------------------------------------------------
14-Mar-2021 14:51:23 - Running job #1
------------------------------------------------------------------------
14-Mar-2021 14:51:23 - Running 'fMRI model specification (design only)'
14-Mar-2021 14:51:23 - Failed  'fMRI model specification (design only)'
questdlg is not available in this version of Octave
In file "/usr/share/octave/4.2.2/m/gui/questdlg.m" (???), function "questdlg" at line 123.
In file "/home/remi/matlab/SPM/spm12/spm_input.m" (v6510), function "spm_input" at line 1234.
In file "/home/remi/matlab/SPM/spm12/config/spm_run_fmri_spec.m" (v6562), function "spm_run_fmri_spec" at line 34.

The following modules did not run:
Failed: fMRI model specification (design only)

error: Job execution failed. The full log of this run can be found in 

Inline plot failed, consider trying another graphics toolkit
error: print: no axes object in figure to print
error: called from
    _make_figures>safe_print at line 125 column 7
    _make_figures at line 49 column 13



In [ ]:
load(fullfile(ouput_dir, 'SPM.mat'))
X1 = SPM.xX.X;

## Block model with HRF

We use the same events but they all last 7 seconds instead of 1.

In [10]:
conditions(1) = set_condition('c1', [30, 70, 100], [7, 7, 7]);
conditions(2) = set_condition('c2', [10, 30, 90], [7, 7, 7]);     
conditions(3) = set_condition('c3', [30, 40, 60], [7, 7, 7]);  

design = set_design(ouput_dir, tr, nb_scans, conditions);
matlabbatch{1}.spm.stats.fmri_design = design;

spm_jobman('run', matlabbatch);
cd(start_dir)

load(fullfile(ouput_dir, 'SPM.mat'))
X2 = SPM.xX.X;



------------------------------------------------------------------------
14-Mar-2021 14:51:25 - Running job #2
------------------------------------------------------------------------
14-Mar-2021 14:51:25 - Running 'fMRI model specification (design only)'
14-Mar-2021 14:51:25 - Failed  'fMRI model specification (design only)'
questdlg is not available in this version of Octave
In file "/usr/share/octave/4.2.2/m/gui/questdlg.m" (???), function "questdlg" at line 123.
In file "/home/remi/matlab/SPM/spm12/spm_input.m" (v6510), function "spm_input" at line 1234.
In file "/home/remi/matlab/SPM/spm12/config/spm_run_fmri_spec.m" (v6562), function "spm_run_fmri_spec" at line 34.

The following modules did not run:
Failed: fMRI model specification (design only)

error: Job execution failed. The full log of this run can be found in MATLAB command window, starting with the lines (look for the line showing the exact #job as displayed in this error message)
------------------ 
Running job #2
----

Inline plot failed, consider trying another graphics toolkit
error: print: no axes object in figure to print
error: called from
    _make_figures>safe_print at line 125 column 7
    _make_figures at line 49 column 13



## Event related model with Finite Impulse Response model

In [11]:
conditions(1) = set_condition('c1', [30, 70, 100], [1, 1, 1]);
conditions(2) = set_condition('c2', [10, 30, 90], [1, 1, 1]);     
conditions(3) = set_condition('c3', [30, 40, 60], [1, 1, 1]);  

design = set_design(ouput_dir, tr, nb_scans, conditions);

design = rmfield(design.bases, 'hrf');
design.bases.fir.length = 32;
design.bases.fir.order = 3;

matlabbatch{1}.spm.stats.fmri_design = design;

spm_jobman('run', matlabbatch);
cd(start_dir)

load(fullfile(ouput_dir, 'SPM.mat'))
X3 = SPM.xX.X;

error: No executable modules, but still unresolved dependencies or incomplete module inputs.
error: called from
    spm_jobman>fill_run_job at line 472 column 5
    spm_jobman at line 247 column 13


# Peek under the hood

In [12]:
frame_times = zeros(nb_scans * tr, 1);  % here are the correspoding frame times


In [13]:
% Gives the shape of the HRF with a sampling frequency of 1 Hz
xBF.dt = 1; 
xBF.name = 'hrf';
xBF.T = 1;
SPM.xBF = spm_get_bf(xBF);

A lot of the code for this was extrracted from `spm_fMRI_design` that usually takes care of setting up the fMRI design.

In [14]:
    %-Resample regressors at acquisition times (32 bin offset)
    %----------------------------------------------------------------------
    if ~isempty(X)
        X = X((0:(k - 1))*fMRI_T + fMRI_T0 + 32,:);
    end
    
    %-Orthogonalise (within trial type)
    %----------------------------------------------------------------------
    for i = 1:length(Fc)
        if i<= numel(U) && ... % for Volterra kernels
                (~isfield(U(i),'orth') || U(i).orth)
            p = ones(size(Fc(i).i));
        else
            p = Fc(i).p;
        end
        for j = 1:max(p)
            X(:,Fc(i).i(p==j)) = spm_orth(X(:,Fc(i).i(p==j)));
        end
    end

error: 'X' undefined near line 3 column 17
error: 'Fc' undefined near line 3 column 22


In [15]:
    %-Append mean-corrected regressors and names
    %----------------------------------------------------------------------
    X     = [X spm_detrend(C)];
    Xn    = {Xn{:} Cname{:}};

error: 'X' undefined near line 3 column 14
error: 'Xn' undefined near line 1 column 14


In [16]:
run_idx = 1;

SPM.Sess(run_idx).U = conditions;
SPM.nscan(run_idx) = n_scans;

% We prepare the onsets for the convolution
U = spm_get_ons(SPM, run_idx);

% We actually perform the convolution
X1 = spm_Volterra(U, SPM.xBF.bf);

error: 'n_scans' undefined near line 1 column 22
error: structure has no member 'ons'
error: called from
    spm_get_ons at line 79 column 11
error: 'U' undefined near line 2 column 19


In [ ]:
xBF.name = 'Finite Impulse Response';
xBF.length = 32;
xBF.dt = 8;
SPM.xBF = spm_get_bf(xBF);

X3 = spm_Volterra(U, SPM.xBF.bf);

In [ ]:
% block design
xBF.name = 'hrf';
SPM.xBF = spm_get_bf(xBF);

for iCdt = 1:numel(conditions)
    SPM.Sess(run_idx).U(iCdt).dur = ones(size(conditions(iCdt).ons)) * 7;
end
U = spm_get_ons(SPM, run_idx);
X2 = spm_Volterra(U, SPM.xBF.bf);

In [ ]:
subplot(131)

%X1 = [X1(33:end, :), motion];
imagesc(X1)

x_tick_label = cat(1, cat(1,conditions.name), add_reg_names');

set(gca, 'xtick', 1:numel(x_tick_label), ...
         'xticklabel', x_tick_label, ...
         'fontsize', 10)
         
subplot(132)

%X2 = [X2(33:end, :), motion];
imagesc(X2)

x_tick_label = cat(1, cat(1,conditions.name), add_reg_names');

set(gca, 'xtick', 1:numel(x_tick_label), ...
         'xticklabel', x_tick_label, ...
         'fontsize', 10)         
         
subplot(133)        
%X2 = [X2(33:end, :), motion];
imagesc(X3)

% x_tick_label = cat(1, cat(1,conditions.name), add_reg_names');

% set(gca, 'xtick', 1:numel(x_tick_label), ...
%         'xticklabel', x_tick_label, ...
%         'fontsize', 10)         